For changing formats and wrangling training data.

In [ ]:
import csv
import utils

In [ ]:
import pathlib

In [ ]:
exp_dir = pathlib.Path("..")

Turning our own matches into same format as given:

In [ ]:
index = 0
for fn in ("matches", "notmatches"):
    with (exp_dir / (fn + ".csv")).open("r") as fin, \
        (utils.datadir / "links" / (fn + ".csv")).open("w") as fout:
        reader = csv.reader(fin)
        writer = csv.writer(fout, delimiter="|")
        writer.writerow(["linkID", "kilde", "kipnr", "lbnr",
                         "tabel", "navn", "faarb"])
        while True:
            try:
                a = next(reader)
                b = next(reader)
                linkID = "a" + str(index)
                for row in (a,b):
                    ft = row[1]
                    tabel = row[2]
                    kip = row[-2]
                    løb = row[-1]
                    if "," in løb:
                        løb = løb.split(",")[0]
                    navn = row[5]
                    fødeår = row[8]
                    writer.writerow([linkID, ft, kip, løb,
                                    tabel, navn, fødeår])
                index += 1
            except StopIteration:
                break

Reading from such a file into something (being reconstructed presently)...

In [ ]:
relevant = set((1845, 1850, 1860, 1880, 1885))

def buildLinksDict(fd):
    reader = csv.reader(fd, delimiter="|")
    next(reader) # skip header
    links = {year: {} for year in relevant} # year -> (kip,løb) -> linkID
    for row in reader:
        year = utils.extractYear(row[1])
        if year is None:
            continue
        if year not in relevant:
            continue
        kipløb = (row[2], int(row[3]))
        links[year][kipløb] = row[0]
    return links

In [ ]:
with (utils.datadir / "links" / "matches.csv").open("r") as fd:
    d = buildLinksDict(fd)